In [29]:
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
datos = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")
datos.head(10)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II
5,Male,29.0,1.62,53.0,no,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Automobile,Normal_Weight
6,Female,23.0,1.50,55.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,0.0,Sometimes,Motorbike,Normal_Weight
7,Male,22.0,1.64,53.0,no,no,2.0,3.0,Sometimes,no,2.0,no,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
8,Male,24.0,1.78,64.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,1.0,Frequently,Public_Transportation,Normal_Weight
9,Male,22.0,1.72,68.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,1.0,1.0,no,Public_Transportation,Normal_Weight


In [30]:
rng = np.random.RandomState(seed=42)
datos["random_cat"] = rng.randint(3, size=datos.shape[0])
datos["random_num"] = rng.randn(datos.shape[0])
categorical_columns = ["Gender", "family_history_with_overweight", "FAVC", "CAEC","SMOKE", "SCC","CALC","MTRANS","random_cat"]
numerical_columns = ["Age", "FCVC", "NCP", "CH2O", "FAF","TUE","random_num"]
X = datos[categorical_columns + numerical_columns]
y = datos["NObeyesdad"]
y = datos['NObeyesdad'].map({'Normal_Weight': 0, 'Insufficient_Weight': 1, 'Obesity_Type_I':2,'Obesity_Type_II':3, 'Obesity_Type_III':4, 'Overweight_Level_I':5,'Overweight_Level_II':6})
yname = ["NObeyesdad"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

0       0
1       0
2       0
3       5
4       6
       ..
2106    4
2107    4
2108    4
2109    4
2110    4
Name: NObeyesdad, Length: 2111, dtype: int64

In [14]:
X_train.head(10)

,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,CH2O,SCC,CALC,MTRANS,random_cat,Age,FCVC,NCP,CH2O,FAF,TUE,random_num
1007,Male,yes,yes,Sometimes,no,2.000000,no,Sometimes,Public_Transportation,1,22.943130,1.882235,1.240046,2.000000,0.000000,1.541493,-0.147780
822,Female,yes,yes,Sometimes,no,2.597746,no,Sometimes,Public_Transportation,1,21.938831,1.906194,3.987707,2.597746,1.922234,1.376124,1.173125
688,Male,no,yes,Sometimes,no,2.357520,no,Sometimes,Public_Transportation,1,17.729923,2.051283,2.645858,2.357520,0.291309,0.897942,0.123205
836,Female,yes,yes,Sometimes,no,1.771781,no,Sometimes,Automobile,2,31.255587,2.615788,3.000000,1.771781,0.759422,0.686529,-0.787784
811,Male,yes,yes,Sometimes,no,2.177386,no,Sometimes,Public_Transportation,1,19.741202,2.000000,3.000000,2.177386,0.336814,0.004813,0.006422
1430,Male,yes,yes,Sometimes,no,2.856521,no,Sometimes,Automobile,0,37.936044,2.045160,1.874532,2.856521,2.005286,0.105408,0.481866
1751,Male,yes,yes,Sometimes,no,2.902682,no,Sometimes,Public_Transportation,0,26.199321,2.423291,3.000000,2.902682,1.243567,0.555591,-0.924466
610,Female,yes,yes,Frequently,no,2.975887,no,no,Public_Transportation,2,21.798856,2.979383,3.000000,2.975887,0.945093,1.241755,0.586694
718,Female,yes,yes,Frequently,no,1.147121,no,no,Public_Transportation,0,21.310907,2.919584,3.697831,1.147121,0.993058,0.089220,1.076007
283,Male,no,no,Sometimes,no,3.000000,no,Sometimes,Walking,1,17.000000,2.000000,3.000000,3.000000,2.000000,1.000000,-0.973069


In [33]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler

categorical_encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1, encoded_missing_value=-1
)
numerical_pipe = StandardScaler()

preprocessing = ColumnTransformer(
    [
        ("Codificación de Categóricas", categorical_encoder, categorical_columns),
        ("Transformación de numéricas", numerical_pipe, numerical_columns),
    ],
    verbose_feature_names_out=False,
)

random_forest = RandomForestClassifier(random_state=42)

flujo = Pipeline(
    [
        ("Preprocesamiento", preprocessing),
        ("Clasificador", random_forest),
    ]
)
flujo.fit(X_train, y_train)

Pipeline(steps=[('Preprocesamiento',
                 ColumnTransformer(transformers=[('Codificación de Categóricas',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['Gender',
                                                   'family_history_with_overweight',
                                                   'FAVC', 'CAEC', 'SMOKE',
                                                   'SCC', 'CALC', 'MTRANS',
                                                   'random_cat']),
                                                 ('Transformación de numéricas',
                                                  StandardScaler(),
                                                  ['Age', 'FCVC', 'NCP', 'CH2O',
                                                   'FAF', 'TUE',
                                                   'random_num'])],
                                   verbose_feature_names_out=False)),
                ('Clasificador', RandomForestClassifier(random_state=42))])